In [1]:
!pip3 install gensim
!pip3 install fuzzywuzzy
!pip3 install textblob

spacy 2.0.12 has requirement cymem<1.32,>=1.30, but you'll have cymem 2.0.2 which is incompatible.
spacy 2.0.12 has requirement murmurhash<0.29,>=0.28, but you'll have murmurhash 1.0.1 which is incompatible.
spacy 2.0.12 has requirement preshed<2.0.0,>=1.0.0, but you'll have preshed 2.0.1 which is incompatible.
spacy 2.0.12 has requirement regex==2017.4.5, but you'll have regex 2017.11.9 which is incompatible.
spacy 2.0.12 has requirement thinc<6.11.0,>=6.10.3, but you'll have thinc 6.12.0 which is incompatible.
thinc 6.12.0 has requirement msgpack-numpy<0.4.4.0, but you'll have msgpack-numpy 0.4.4.1 which is incompatible.


thinc 6.12.0 has requirement msgpack-numpy<0.4.4.0, but you'll have msgpack-numpy 0.4.4.1 which is incompatible.
spacy 2.0.12 has requirement cymem<1.32,>=1.30, but you'll have cymem 2.0.2 which is incompatible.
spacy 2.0.12 has requirement murmurhash<0.29,>=0.28, but you'll have murmurhash 1.0.1 which is incompatible.
spacy 2.0.12 has requirement preshed<2.0.0,>=1.0.0, but you'll have preshed 2.0.1 which is incompatible.
spacy 2.0.12 has requirement regex==2017.4.5, but you'll have regex 2017.11.9 which is incompatible.
spacy 2.0.12 has requirement thinc<6.11.0,>=6.10.3, but you'll have thinc 6.12.0 which is incompatible.


thinc 6.12.0 has requirement msgpack-numpy<0.4.4.0, but you'll have msgpack-numpy 0.4.4.1 which is incompatible.
spacy 2.0.12 has requirement cymem<1.32,>=1.30, but you'll have cymem 2.0.2 which is incompatible.
spacy 2.0.12 has requirement murmurhash<0.29,>=0.28, but you'll have murmurhash 1.0.1 which is incompatible.
spacy 2.0.12 has requirement preshed<2.0.0,>=1.0.0, but you'll have preshed 2.0.1 which is incompatible.
spacy 2.0.12 has requirement regex==2017.4.5, but you'll have regex 2017.11.9 which is incompatible.
spacy 2.0.12 has requirement thinc<6.11.0,>=6.10.3, but you'll have thinc 6.12.0 which is incompatible.


In [ ]:
import os
from gensim.similarities import WmdSimilarity
import pandas as pd
import numpy as np
import gensim

from gensim import corpora
import gensim.downloader as api
from gensim.matutils import softcossim
from gensim.models import Word2Vec
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
from nltk import word_tokenize
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
import xgboost as xgb
from textblob import Word
from textblob import TextBlob
import spacy

!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')  

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
## Load the Drive helper and mount
#from google.colab import drive
#
## This will prompt for authorization.
#drive.mount('/content/drive')

# **Loading data**

In [ ]:
train_path = os.path.join('..','data','test_data_v3_processed.csv')
train_data = pd.read_csv(train_path)
#train_data = train_data.dropna(how="any").reset_index(drop=True)

In [ ]:
train_data.head()

# Constants

In [ ]:
QUESTION1 = 'question1_lemma'
QUESTION2 = 'question2_lemma'

QUESTION1_original = 'question1'
QUESTION2_original = 'question2'

In [ ]:
## TODO DELETE
#train_data['question1_lemma'] = train_data['question1']
#train_data['question2_lemma'] = train_data['question2']
#train_data = train_data[:10]

# **Fuzzy features**

In [ ]:
def setFuzzyFeatures(df):  
    df['fuzz_ratio'] = df.apply(lambda x: fuzz.ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
    df['fuzz_partial_ratio'] = df.apply(lambda x: fuzz.partial_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
    df['fuzz_partial_token_set_ratio'] = df.apply(lambda x: fuzz.partial_token_set_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
    df['fuzz_partial_token_sort_ratio'] = df.apply(lambda x: fuzz.partial_token_sort_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
    df['fuzz_token_set_ratio'] = df.apply(lambda x: fuzz.token_set_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)
    df['fuzz_token_sort_ratio'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x[QUESTION1]), str(x[QUESTION2])), axis=1)

# **String features**

In [ ]:
def _avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def _numStopWords(sentence):
    return len([x for x in sentence.split() if x in stop_words])

def _numNumbers(sentence):
    return len([x for x in sentence.split() if x.isdigit()])

def _numUppercaseWords(sentence):
    return len([x for x in sentence.split() if x.isupper()])

def setStringFeatures(df):  
    df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
    df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
    df['diff_len'] = df.len_q1 - df.len_q2
    df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
    df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
    df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
    df['common_words'] = df.apply(lambda x: len(set(str(x[QUESTION1]).lower().split()).intersection(set(str(x[QUESTION2]).lower().split()))), axis=1)
    df['avg_word_q1'] = df.question1.apply(lambda x: _avg_word(x))
    df['avg_word_q2'] = df.question2.apply(lambda x: _avg_word(x))
    df['num_stop_words_q1'] = df.question1.apply(lambda x: _numStopWords(x))
    df['num_stop_words_q2'] = df.question2.apply(lambda x: _numStopWords(x))
    df['numerics_q1'] = df.question1.apply(lambda x: _numNumbers(x))
    df['numerics_q2'] = df.question2.apply(lambda x: _numNumbers(x))
    df['uppercase_q1'] = df.question1.apply(lambda x: _numUppercaseWords(x))
    df['uppercase_q2'] = df.question2.apply(lambda x: _numUppercaseWords(x))
  

# **Word2Vec features**

In [ ]:
w2v_model = api.load("word2vec-google-news-300")

In [ ]:
def _getDictionary(df):  
    documents = list(df[QUESTION1_original].apply(lambda x: x.split()))+list(df[QUESTION2_original].apply(lambda x: x.split()))
    
    return corpora.Dictionary(documents)
   
def _softCossim(row, dictionary, similarity_matrix):
    q1 = row[QUESTION1_original].split()
    q2 = row[QUESTION2_original].split()
    
    q1 = dictionary.doc2bow(q1)
    q2 = dictionary.doc2bow(q2)
    
    return softcossim(q1, q2, similarity_matrix)
 
def setWord2VecFeatures(df):
    dictionary = _getDictionary(df)
    similarity_matrix = w2v_model.similarity_matrix(dictionary)
    
    df['softcossim'] = df.apply(lambda row: _softCossim(row, dictionary, similarity_matrix), axis=1)
  

# **TextBlob**

In [ ]:
def _numSpellingMistakes(sentence):
    mistakes = 0
    for word in sentence.split():
    if TextBlob(word).correct() != word:
        mistakes += 1
    return mistakes


def setTextBlobFeatures(df):
    df['mistakes_q1'] = df.question1.apply(lambda x: _numSpellingMistakes(x))
    df['mistakes_q2'] = df.question2.apply(lambda x: _numSpellingMistakes(x))

# **Spacy**

In [ ]:
def _getSpacySimilarity(row):
    q1 = row[QUESTION1_original]
    q2 = row[QUESTION2_original]
    
    tokens1 = nlp(Q1)
    tokens2 = nlp(Q2)
    
    return tokens1.similarity(tokens2)

def setSpacyFeatures(df):
    df['spacy_sim'] = df.apply(lambda row: _getSpacySimilarity(row), axis=1)
  

# Merge Features **Functions**

In [ ]:
import datetime

def saveDf(df):
    df.set_index('test_id', inplace=True)
    df.to_csv(path_or_buf=train_path, sep=',')


def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60.
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

def setFeatures(df):
    print('start setting features')
    
    #start_time = datetime.datetime.now()
    #setFuzzyFeatures(df)
    #end_time = datetime.datetime.now()
    #seconds_elapsed = (end_time - start_time).total_seconds()
    #print('finished setFuzzyFeatures')
    #print("It took {} to execute this".format(hms_string(seconds_elapsed)))
    #
    #start_time = datetime.datetime.now()
    #setStringFeatures(df)
    #end_time = datetime.datetime.now()
    #seconds_elapsed = (end_time - start_time).total_seconds()
    #print('finished setStringFeatures')
    #print("It took {} to execute this".format(hms_string(seconds_elapsed)))
    #
    start_time = datetime.datetime.now()
    setWord2VecFeatures(df)
    end_time = datetime.datetime.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    print('finished setWord2VecFeatures')
    print("It took {} to execute this".format(hms_string(seconds_elapsed)))
    
    start_time = datetime.datetime.now()
    setTextBlobFeatures(df)
    end_time = datetime.datetime.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    print('finished setTextBlobFeatures')
    print("It took {} to execute this".format(hms_string(seconds_elapsed)))
    
    start_time = datetime.datetime.now()
    setSpacyFeatures(df)
    end_time = datetime.datetime.now()
    seconds_elapsed = (end_time - start_time).total_seconds()
    print('finished setSpacyFeatures')
    print("It took {} to execute this".format(hms_string(seconds_elapsed)))
  

# **Execution**

In [ ]:
df = train_data#.copy()

In [ ]:
setFeatures(df)

In [ ]:
df.head()

In [ ]:
saveDf(df)